# Building a Synthesizer for Pivot

In this tutorial, we will build a simple but efficient synthesizer for the Pandas [pivot](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.DataFrame.pivot.html) function. More concretely, given an input dataframe and a desired output dataframe, our synthesizer will output the arguments that need to be passed to pivot that can reproduce the desired output.

## Arguments Generator for Pivot
First let us define a generator that will be our synthesis engine for pivot. As mentioned earlier, this generator will enumerate possible argument combinations. Here is one version which simply selects one of the columns for each of the arguments (or the default value of `None`).

In [1]:
from atlas import generator

@generator
def pivot_args_generator(input_df):
    arg_columns = Select([None] + list(input_df.columns))
    arg_index = Select([None] + list(input_df.columns))
    arg_values = Select([None] + list(input_df.columns))
    
    return {'index': arg_index, 'columns': arg_columns, 'values': arg_values}

Now let's try running it on a sample dataframe

In [2]:
import pandas as pd
df = pd.DataFrame({
  'foo': ['one', 'one', 'one', 'two', 'two', 'two'],
  'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
  'baz': [10, 20, 30, 40, 50, 60],
})
df

,foo,bar,baz
0,one,A,10
1,one,B,20
2,one,C,30
3,two,A,40
4,two,B,50
5,two,C,60


In [3]:
for args in pivot_args_generator.generate(df):
    print(args)

{'index': None, 'columns': None, 'values': None}
{'index': None, 'columns': None, 'values': 'foo'}
{'index': None, 'columns': None, 'values': 'bar'}
{'index': None, 'columns': None, 'values': 'baz'}
{'index': 'foo', 'columns': None, 'values': None}
{'index': 'foo', 'columns': None, 'values': 'foo'}
{'index': 'foo', 'columns': None, 'values': 'bar'}
{'index': 'foo', 'columns': None, 'values': 'baz'}
{'index': 'bar', 'columns': None, 'values': None}
{'index': 'bar', 'columns': None, 'values': 'foo'}
{'index': 'bar', 'columns': None, 'values': 'bar'}
{'index': 'bar', 'columns': None, 'values': 'baz'}
{'index': 'baz', 'columns': None, 'values': None}
{'index': 'baz', 'columns': None, 'values': 'foo'}
{'index': 'baz', 'columns': None, 'values': 'bar'}
{'index': 'baz', 'columns': None, 'values': 'baz'}
{'index': None, 'columns': 'foo', 'values': None}
{'index': None, 'columns': 'foo', 'values': 'foo'}
{'index': None, 'columns': 'foo', 'values': 'bar'}
{'index': None, 'columns': 'foo', 'value

However, not all the argument combinations printed above are valid. Not convinced? Let's try executing them.

In [4]:
for args in pivot_args_generator.generate(df):
    print(args, df.pivot(**args))

ValueError: cannot label index with a null key

Pandas threw an error as our generator is not *precise* while enumerating arguments. That is, it generates argument combinations which cause the pandas pivot function to throw an error. There are a number of hard-to-see constraints embedded in the [documentation](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.DataFrame.pivot.html) for pivot. For example, the columns argument is not marked *Optional* even though it is a keyword argument. We have already done the job of incorporating all these constraints in the generator below.

In [5]:
@generator
def pivot_args_generator(input_df): 
    def dup_filter(cand):
        try:
            return not any(input_df[[cand, arg_columns]].duplicated())
        except:
            return True

    arg_columns = Select(input_df.columns)
    arg_index = Select([None] + list(filter(dup_filter, set(input_df.columns) - {arg_columns})))

    if input_df.index.nlevels > 1 and arg_index is None:
        arg_values = None
    else:
        arg_values = Select(set(input_df.columns) | {None})

    return {'columns': arg_columns, 'index': arg_index, 'values': arg_values}

In [6]:
for args in pivot_args_generator.generate(df):
    print(args)

{'columns': 'foo', 'index': None, 'values': 'baz'}
{'columns': 'foo', 'index': None, 'values': 'foo'}
{'columns': 'foo', 'index': None, 'values': None}
{'columns': 'foo', 'index': None, 'values': 'bar'}
{'columns': 'foo', 'index': 'baz', 'values': 'baz'}
{'columns': 'foo', 'index': 'baz', 'values': 'foo'}
{'columns': 'foo', 'index': 'baz', 'values': None}
{'columns': 'foo', 'index': 'baz', 'values': 'bar'}
{'columns': 'foo', 'index': 'bar', 'values': 'baz'}
{'columns': 'foo', 'index': 'bar', 'values': 'foo'}
{'columns': 'foo', 'index': 'bar', 'values': None}
{'columns': 'foo', 'index': 'bar', 'values': 'bar'}
{'columns': 'bar', 'index': None, 'values': 'baz'}
{'columns': 'bar', 'index': None, 'values': 'foo'}
{'columns': 'bar', 'index': None, 'values': None}
{'columns': 'bar', 'index': None, 'values': 'bar'}
{'columns': 'bar', 'index': 'baz', 'values': 'baz'}
{'columns': 'bar', 'index': 'baz', 'values': 'foo'}
{'columns': 'bar', 'index': 'baz', 'values': None}
{'columns': 'bar', 'index

## Building a Brute-Force Synthesizer
Given `pivot_args_generator` we are now ready to build our first brute-force synthesizer for pivot. Given an input-output pair, we will simply enumerate and execute all argument combinations till we find the right one.

In [7]:
from atlas.synthesis.pandas.checker import Checker
def synthesize(input_df, output_df):
    for args in pivot_args_generator.generate(input_df):
        print("Trying Combination:", args)
        result = input_df.pivot(**args)
        if Checker.check(result, output_df):
            print("Solution Found:", args)
            break

Try it out!

In [8]:
desired_output = pd.DataFrame({'one': {'A': 10, 'B': 20, 'C': 30}, 'two': {'A': 40, 'B': 50, 'C': 60}})
desired_output

,one,two
A,10,40
B,20,50
C,30,60


In [9]:
synthesize(df, desired_output)

Trying Combination: {'columns': 'foo', 'index': None, 'values': 'baz'}
Trying Combination: {'columns': 'foo', 'index': None, 'values': 'foo'}
Trying Combination: {'columns': 'foo', 'index': None, 'values': None}
Trying Combination: {'columns': 'foo', 'index': None, 'values': 'bar'}
Trying Combination: {'columns': 'foo', 'index': 'baz', 'values': 'baz'}
Trying Combination: {'columns': 'foo', 'index': 'baz', 'values': 'foo'}
Trying Combination: {'columns': 'foo', 'index': 'baz', 'values': None}
Trying Combination: {'columns': 'foo', 'index': 'baz', 'values': 'bar'}
Trying Combination: {'columns': 'foo', 'index': 'bar', 'values': 'baz'}
Solution Found: {'columns': 'foo', 'index': 'bar', 'values': 'baz'}


## Building a Smart Synthesizer
Our brute-force synthesizer works pretty well, but there's still a lot that can be done. First, notice the number of argument combinations explored before arriving at the solution. For large dataframes the synthesis process may be slow. Second, the generator is not *predictive*. It does not tell us what it thinks are the most promising argument combinations. Rather it just goes over the space of values deterministically.

Let us try to *train* the generator to return argument combinations that are *more likely* to be correct for a given input-output dataframe pair.

### Adding Output to the Generator
You may have noticed that the output dataframe is not visible to the argument generator. In order to build a smarter generator, we need to give it access to the output. 

In [10]:
@generator
def pivot_args_generator(input_df, output_df):
    pass

### Adding Context to the Generator
We will pass a **context** to each of the Select operators that will help them make the the right choices by conditioning on this context. Can you guess what the context will be? That's right, it will simply be the input-output example.

In [11]:
@generator
def pivot_args_generator(input_df, output_df): 
    def dup_filter(cand):
        try:
            return not any(input_df[[cand, arg_columns]].duplicated())
        except:
            return True

    c = {'Input': input_df, 'Output': output_df}
    arg_columns = Select(input_df.columns, context=c)
    arg_index = Select([None] + list(filter(dup_filter, set(input_df.columns) - {arg_columns})), context=c)

    if input_df.index.nlevels > 1 and arg_index is None:
        arg_values = None
    else:
        arg_values = Select(set(input_df.columns) | {None}, context=c)

    return {'columns': arg_columns, 'index': arg_index, 'values': arg_values}

Need to modify the `synthesize` procedure a bit as well to sync with the above changes

In [12]:
def synthesize(input_df, output_df):
    for args in pivot_args_generator.generate(input_df, output_df):
        print("Trying Combination:", args)
        result = input_df.pivot(**args)
        if Checker.check(result, output_df):
            print("Solution Found:", args)
            break

## Training the Generator

### Defining the Model
We need to define the predictive models that the generator will use to refine its output. In particular, we need to define the models used by the Select operators.

Our overall generator model uses a single Tensorflow Graph-Neural-Network architecture for all the Select operators.

*TODO : Construct graph encoding inside the notebook, and use a PyTorch model using PyTorch Geometric*

In [13]:
from typing import Mapping, Any, Collection
from atlas.tracing import OpTrace
from atlas.operators import operator
from atlas.models.catalogue import Models
from atlas.models.tensorflow.graphs.operators import SelectGGNN
from atlas.synthesis.pandas.encoders import PandasGraphEncoder

class PivotGeneratorModel(Models.Generators.Imitation.IndependentOperators):
    @operator
    def Select(*args, **kwargs):
        config = {
            'learning_rate': 0.01,
            'node_dimension': 50,
            'classifier_hidden_dims': [50],
            'batch_size': 30000,
            'layer_timesteps': [1, 1, 1]
        }
        
        return PivotSelectModel(config)
    
class PivotSelectModel(SelectGGNN):
    def __init__(self, params: Mapping[str, Any]):
        self.encoder = PandasGraphEncoder()
        params.update({
            'num_node_features': self.encoder.get_num_node_features(),
            'num_edge_types': self.encoder.get_num_edge_types()
        })
        
        super().__init__(params)
        
    def train(self, training_data: Collection[OpTrace], validation_data: Collection[OpTrace], *args, **kwargs):
        encoded_train = [self.encoder.Select(domain=op.domain, 
                                             context=op.context, 
                                             choice=op.choice, 
                                             sid=op.op_info.sid) for op in training_data]
        encoded_valid = [self.encoder.Select(domain=op.domain, 
                                             context=op.context, 
                                             choice=op.choice, 
                                             sid=op.op_info.sid) for op in validation_data]
        super().train(encoded_train, encoded_valid, *args, **kwargs)
        
    def infer(self, domain, context: Any = None, sid: str = '', **kwargs):
        encoding = self.encoder.Select(domain, context, mode='inference', sid=sid)
        inference = super().infer([encoding])[0]
        return [val for val, prob in sorted(inference, key=lambda x: -x[1])]


### Gather Training Data
To gather training data, we will simply take three dataframes, enumerate all possible argument combinations, execute them and collect the output. We will enable tracing to keep track of the operator choices made. We will then retroactively modify the trace to incorporate the resulting output in the context.

In [14]:
dataframes = [
    pd.DataFrame({
        'foo': ['one', 'one', 'one', 'two', 'two', 'two'],
        'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
        'baz': [10, 20, 30, 40, 50, 60],
    }),

    pd.DataFrame({
        'date': ['1959-03-31 00:00:00'] * 3 + ['1959-06-30 00:00:00'] * 3 + [
            '1959-09-30 00:00:00'] * 3 + ['1959-12-31 00:00:00'],
        'item': ['realgdp', 'infl', 'unemp', 'realgdp', 'infl', 'unemp', 'realgdp', 'infl',
                 'unemp', 'realgdp'],
        'value': [2710.349, 0.001, 5.800, 2778.801, 2.340, 5.100, 2775.488, 2.740, 5.300,
                  2785.204],
    }),

    pd.DataFrame({
        'Date': ['2018-02-18', '2018-02-24', '2018-02-24', '2018-03-01', '2018-03-11', '2018-03-11'],
        'Category': ['Social', 'Lunch', 'Social', 'Lunch', 'Lunch', 'Brunch'],
        'Location': ['Terrace', "Pox", "Gate 320", "Pox", "Cola 543", "Terrace"],
        'Expense': [98.34, 245.63, 121.89, 248.00, 240.40, 561.48],
        'Balance': [9971.66, 9726.03, 9604.14, 9356.04, 9115.74, 8554.26]
    })
]

In [15]:
traces = []
input_outputs = []
for df in dataframes[:3]:
    for args, trace in pivot_args_generator.with_env(tracing=True).generate(df, None):
        output = df.pivot(**args)
        input_outputs.append((df, output))
        traces.append(trace)
        
# Need to replay as in the previous run as the output was None and 
# consequently the context recorded was incorrect
# The [1] at the end simply extracts the trace part of the resulting tuple
traces = [pivot_args_generator.with_env(replay=trace, tracing=True).call(i, o)[1]
          for (i, o), trace in zip(input_outputs, traces)]

### Train the Model
Let us start training the model

In [16]:
model = PivotGeneratorModel()
model.train(traces, traces, num_epochs=-1)

100%|██████████| 210/210 [00:00<00:00, 241647.14it/s]


[+] Training model for OpInfo(sid='/pivot_args_generator/Select@@1', gen_name='pivot_args_generator', op_type='Select', index=1, gen_group=None, uid=None, tags=None)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/usr/local/lib/python3.7/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[Training(1/-1)] Loss:  1.431255 Accuracy:  0.3238
[Validation(1/-1)] Loss:  1.412953 Accuracy:  0.3286
[Training(2/-1)] Loss:  1.412932 Accuracy:  0.3286
[Validation(2/-1)] Loss:  1.373543 Accuracy:  0.3286
[Training(3/-1)] Loss:  1.371705 Accuracy:  0.3524
[Validation(3/-1)] Loss:  1.311079 Accuracy:  0.3286
[Training(4/-1)] Loss:  1.309602 Accuracy:  0.3286
[Validation(4/-1)] Loss:  1.270060 Accuracy:  0.3286
[Training(5/-1)] Loss:  1.270310 Accuracy:  0.3286
[Validation(5/-1)] Loss:  1.262313 Accuracy:  0.3286
[Training(6/-1)] Loss:  1.262292 Accuracy:  0.3286
[Validation(6/-1)] Loss:  1.261103 Accuracy:  0.3286
[Training(7/-1)] Loss:  1.261276 Accuracy:  0.3286
[Validation(7/-1)] Loss:  1.261362 Accuracy:  0.3286
[Training(8/-1)] Loss:  1.261232 Accuracy:  0.3286
[Validation(8/-1)] Loss:  1.261296 Accuracy:  0.3286
[Training(9/-1)] Loss:  1.261195 Accuracy:  0.3286
[Validation(9/-1)] Loss:  1.260799 Accuracy:  0.3286
[Training(10/-1)] Loss:  1.260785 Accuracy:  0.3286
[Validation(

[Training(17/-1)] Loss:  1.261789 Accuracy:  0.5048
[Validation(17/-1)] Loss:  1.241260 Accuracy:  0.4905
[Training(18/-1)] Loss:  1.228849 Accuracy:  0.4857
[Validation(18/-1)] Loss:  1.212222 Accuracy:  0.4857
[Training(19/-1)] Loss:  1.214649 Accuracy:  0.4714
[Validation(19/-1)] Loss:  1.220903 Accuracy:  0.4381
[Training(20/-1)] Loss:  1.213891 Accuracy:  0.4714
[Validation(20/-1)] Loss:  1.239020 Accuracy:  0.2571
[Training(21/-1)] Loss:  1.268777 Accuracy:  0.2571
[Validation(21/-1)] Loss:  1.200675 Accuracy:  0.5238
[Training(22/-1)] Loss:  1.204381 Accuracy:  0.5619
[Validation(22/-1)] Loss:  1.220874 Accuracy:  0.5524
[Training(23/-1)] Loss:  1.272282 Accuracy:  0.5429
[Validation(23/-1)] Loss:  1.215011 Accuracy:  0.5857
[Training(24/-1)] Loss:  1.214239 Accuracy:  0.5762
[Validation(24/-1)] Loss:  1.196992 Accuracy:  0.5857
[Training(25/-1)] Loss:  1.196399 Accuracy:  0.5810
[Validation(25/-1)] Loss:  1.183847 Accuracy:  0.5857
[Training(26/-1)] Loss:  1.180562 Accuracy:  0

[Training(1/-1)] Loss:  1.650651 Accuracy:  0.1667
[Validation(1/-1)] Loss:  1.653702 Accuracy:  0.1952
[Training(2/-1)] Loss:  1.653650 Accuracy:  0.1952
[Validation(2/-1)] Loss:  1.652600 Accuracy:  0.1952
[Training(3/-1)] Loss:  1.652591 Accuracy:  0.1952
[Validation(3/-1)] Loss:  1.652664 Accuracy:  0.1810
[Training(4/-1)] Loss:  1.652665 Accuracy:  0.1857
[Validation(4/-1)] Loss:  1.652668 Accuracy:  0.3190
[Training(5/-1)] Loss:  1.652669 Accuracy:  0.2238
[Validation(5/-1)] Loss:  1.652645 Accuracy:  0.3095
[Training(6/-1)] Loss:  1.652621 Accuracy:  0.2095
[Validation(6/-1)] Loss:  1.652637 Accuracy:  0.2524
[Training(7/-1)] Loss:  1.652626 Accuracy:  0.2524
[Validation(7/-1)] Loss:  1.652598 Accuracy:  0.1952
[Training(8/-1)] Loss:  1.652587 Accuracy:  0.1952
[Validation(8/-1)] Loss:  1.652548 Accuracy:  0.1952
[Training(9/-1)] Loss:  1.652546 Accuracy:  0.1952
[Validation(9/-1)] Loss:  1.652451 Accuracy:  0.1952
[Training(10/-1)] Loss:  1.652455 Accuracy:  0.1952
[Validation(

[Validation(78/-1)] Loss:  1.556625 Accuracy:  0.1952
[Training(79/-1)] Loss:  1.555783 Accuracy:  0.1952
[Validation(79/-1)] Loss:  1.554919 Accuracy:  0.1952
[Training(80/-1)] Loss:  1.554548 Accuracy:  0.1952
[Validation(80/-1)] Loss:  1.548389 Accuracy:  0.1952
[Training(81/-1)] Loss:  1.547685 Accuracy:  0.1952
[Validation(81/-1)] Loss:  1.543307 Accuracy:  0.1952
[Training(82/-1)] Loss:  1.543500 Accuracy:  0.1952
[Validation(82/-1)] Loss:  1.536313 Accuracy:  0.1952
[Training(83/-1)] Loss:  1.537131 Accuracy:  0.1952
[Validation(83/-1)] Loss:  1.529290 Accuracy:  0.1952
[Training(84/-1)] Loss:  1.529503 Accuracy:  0.1952
[Validation(84/-1)] Loss:  1.526730 Accuracy:  0.2143
[Training(85/-1)] Loss:  1.526695 Accuracy:  0.2143
[Validation(85/-1)] Loss:  1.522706 Accuracy:  0.2143
[Training(86/-1)] Loss:  1.521967 Accuracy:  0.2333
[Validation(86/-1)] Loss:  1.515205 Accuracy:  0.2286
[Training(87/-1)] Loss:  1.516016 Accuracy:  0.2190
[Validation(87/-1)] Loss:  1.521803 Accuracy: 

[Training(155/-1)] Loss:  0.104505 Accuracy:  1.0000
[Validation(155/-1)] Loss:  0.038805 Accuracy:  1.0000
[Training(156/-1)] Loss:  0.066860 Accuracy:  1.0000
[Validation(156/-1)] Loss:  0.034072 Accuracy:  1.0000
[Training(157/-1)] Loss:  0.077655 Accuracy:  1.0000
[Validation(157/-1)] Loss:  0.031574 Accuracy:  1.0000
[Training(158/-1)] Loss:  0.029509 Accuracy:  1.0000
[Validation(158/-1)] Loss:  0.029299 Accuracy:  1.0000
[Training(159/-1)] Loss:  0.040256 Accuracy:  1.0000
[Validation(159/-1)] Loss:  0.026666 Accuracy:  1.0000
[Training(160/-1)] Loss:  0.029731 Accuracy:  1.0000
[Validation(160/-1)] Loss:  0.024596 Accuracy:  1.0000
[Training(161/-1)] Loss:  0.028877 Accuracy:  1.0000
[Validation(161/-1)] Loss:  0.022820 Accuracy:  1.0000
[Training(162/-1)] Loss:  0.049907 Accuracy:  1.0000
[Validation(162/-1)] Loss:  0.020766 Accuracy:  1.0000
[Training(163/-1)] Loss:  0.023560 Accuracy:  1.0000
[Validation(163/-1)] Loss:  0.019606 Accuracy:  1.0000
[Training(164/-1)] Loss:  0.

In [17]:
from atlas.models.utils import save_model
save_model(model, "./trained_model.zip")

## Putting it All Together

Let's assign the model we just trained to our original generator

In [18]:
from atlas.models.utils import restore_model
model = restore_model("./trained_model.zip")
pivot_args_generator.set_default_model(model)

/usr/local/lib/python3.7/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from /var/folders/s9/__w2d9dx2ljdx9qk865hh5qs559bh9/T/tmpka363tl9/models/pivot_args_generator/Select@@1/model.weights
INFO:tensorflow:Restoring parameters from /var/folders/s9/__w2d9dx2ljdx9qk865hh5qs559bh9/T/tmpka363tl9/models/pivot_args_generator/Select@@2/model.weights
INFO:tensorflow:Restoring parameters from /var/folders/s9/__w2d9dx2ljdx9qk865hh5qs559bh9/T/tmpka363tl9/models/pivot_args_generator/Select@@3/model.weights


Let's try running our trained generator on an unseen input-output example

In [19]:
unseen_input_df = pd.DataFrame({
    'series': {0: 'A', 1: 'B', 2: 'C', 3: 'A', 4: 'B', 5: 'C', 6: 'A', 7: 'B', 8: 'C', 9: 'A', 10: 'B', 11: 'C', 12: 'A', 13: 'B', 14: 'C'},
    'step': {0: '100', 1: '100', 2: '100', 3: '101', 4: '101', 5: '101', 6: '102', 7: '102', 8: '102', 9: '103', 10: '103', 11: '103', 12: '104', 13: '104', 14: '104'},
    'value': {0: '1000', 1: '1001', 2: '1002', 3: '1003', 4: '1004', 5: '1005', 6: '1006', 7: '1007', 8: '1008', 9: '1009', 10: '1010', 11: '1011', 12: '1012', 13: '1013', 14: '1014'}
})
unseen_input_df

,series,step,value
0,A,100,1000
1,B,100,1001
2,C,100,1002
3,A,101,1003
4,B,101,1004
5,C,101,1005
6,A,102,1006
7,B,102,1007
8,C,102,1008
9,A,103,1009


In [20]:
unseen_output_df = pd.DataFrame({
    'A': {'100': '1000', '101': '1003', '102': '1006', '103': '1009', '104': '1012'},
    'B': {'100': '1001', '101': '1004', '102': '1007', '103': '1010', '104': '1013'},
    'C': {'100': '1002', '101': '1005', '102': '1008', '103': '1011', '104': '1014'}
})
unseen_output_df

,A,B,C
100,1000,1001,1002
101,1003,1004,1005
102,1006,1007,1008
103,1009,1010,1011
104,1012,1013,1014


In [21]:
synthesize(unseen_input_df, unseen_output_df)

Trying Combination: {'columns': 'series', 'index': 'step', 'values': None}
Trying Combination: {'columns': 'series', 'index': 'step', 'values': 'value'}
Solution Found: {'columns': 'series', 'index': 'step', 'values': 'value'}


Success! As you can see, our trained generator found the solution within the first few attempts! What is even more remarkable is that we trained it on a very small amount of data, data that looked very different from the training dataset.